### Integrantes
 1. Luis Carlos Peña
 2. Andrés Obando
 3. Klaus Rodríguez
 4. Alexander Vega

# Project 1

# Used Vehicle Price Prediction

## Introduction

- 1.2 Million listings scraped from TrueCar.com - Price, Mileage, Make, Model dataset from Kaggle: [data](https://www.kaggle.com/jpayne/852k-used-car-listings)
- Each observation represents the price of an used car

# Exercise P1.1 (25%)

## Notebook explaining the modeling process

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import sklearn.model_selection
from sklearn.model_selection import GridSearchCV 
data = pd.read_csv('dataTrain_carListings.csv')
%matplotlib inline

In [2]:
data = pd.read_csv('dataTrain_carListings.csv')
data_test = pd.read_csv('dataTest_carListings.csv', index_col=0)

In [3]:
data_test= data_test
data_test["State2"]=pd.DataFrame(data_test.State.astype("category").cat.codes)
data_test["Make2"]=pd.DataFrame(data_test.Make.astype("category").cat.codes)
data_test["Model2"]=pd.DataFrame(data_test.Model.astype("category").cat.codes)
data_test3=data_test.drop(["Make","State","Model"], axis=1)

In [4]:
# Converting to categorical variables State, Make and Model 
data["State"] = data["State"].astype('category')
data["Make"] = data["Make"].astype('category')
data["Model"] = data["Model"].astype('category')
data["State_cat"] = data["State"].cat.codes
data["Make_cat"] = data["Make"].cat.codes
data["Model_cat"] = data["Model"].cat.codes

# Extract feature and labels
Y = data['Price']
X = data.drop(['Price', 'State', 'Make', 'Model'], axis = 1)

# Convert to numpy arrays
import numpy as np

X = np.array(X)
Y = np.array(Y)

# Training and Testing Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)

## What we did?

Before showing the model that we finally use, we want to mention all the algorithms that we execute:

1. Bagging of Decision Tree. Best score in kaggle: 3.612
2. Random forest. Best score in Kaggle: 3.521
3. XGBoost. Best score in kaggle: 3.458
 

Finally, we used an xgboost because it reported the best performance

### 1. Begin tuning for XGBoost


First, we tune the max_depth and min_child_weight parameters on a range of values. Later, we will refine these two choices with a smaller grid.

In [5]:
objective = "reg:linear"
seed = 100
n_estimators = 150
learning_rate = 0.1
gamma = 0.1
subsample = 0.8
colsample_bytree = 0.8
reg_alpha = 1
reg_lambda = 1
silent = False

parameters = {}
parameters['objective'] = objective
parameters['seed'] = seed
parameters['n_estimators'] = n_estimators
parameters['learning_rate'] = learning_rate
parameters['gamma'] = gamma
parameters['colsample_bytree'] = colsample_bytree
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
parameters['silent'] = silent

scores = []

cv_params = {'max_depth': [8,16,19,24],
             'min_child_weight': [1,3,5,7]
            }
gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        subsample = subsample,
                                        colsample_bytree = colsample_bytree,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent

                                    ),
                    
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 60.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=150, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1,
       reg_lambda=1, scale_pos_weight=1, seed=100, silent=False,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_depth': [8, 16, 19, 24], 'min_child_weight': [1, 3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [6]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'max_depth': 16, 'min_child_weight': 7}


Best parameters {'max_depth': 16, 'min_child_weight': 7}

### 2. Refine for max_depth and min_child_weight with a smaller grid of values

In [7]:
max_depth = gbm.best_params_['max_depth']
min_child_weight = gbm.best_params_['min_child_weight']
parameters['max_depth'] = max_depth
parameters['min_child_weight'] = min_child_weight
scores.append(gbm.best_score_)

cv_params = {'max_depth': [max_depth-1, max_depth, max_depth+1], ### max_depth=[15,16,17]
             'min_child_weight': [min_child_weight-1, min_child_weight-0.5, min_child_weight, min_child_weight+0.5, min_child_weight+1]
            } ### min_child_weight=[6,6.5,7,7.5,8]

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        subsample = subsample,
                                        colsample_bytree = colsample_bytree,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent,
                                        
                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
                    
)

gbm.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 51.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=150, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1,
       reg_lambda=1, scale_pos_weight=1, seed=100, silent=False,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_depth': [15, 16, 17], 'min_child_weight': [6, 6.5, 7, 7.5, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [8]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'max_depth': 15, 'min_child_weight': 7.5}


Best parameters {'max_depth': 15, 'min_child_weight': 7.5}

### 3. Tuning gamma parameter


In [9]:
max_depth = gbm.best_params_['max_depth']
min_child_weight = gbm.best_params_['min_child_weight']
parameters['max_depth'] = max_depth
parameters['min_child_weight'] = min_child_weight
scores.append(gbm.best_score_)

cv_params = {'gamma': [i/10.0 for i in range(1,10,2)]}  ### gamma= [0.1, 0.3, 0.5, 0.7, 0.9]

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        max_depth = max_depth,
                                        min_child_weight = min_child_weight,
                                        learning_rate = learning_rate,
                                        subsample = subsample,
                                        colsample_bytree = colsample_bytree,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent

                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 15.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=7.5, missing=None, n_estimators=150, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1,
       reg_lambda=1, scale_pos_weight=1, seed=100, silent=False,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [10]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'gamma': 0.1}


Best parameters {'gamma': 0.1}

### 4. Set gamma parameter and tune the subsample and colsample_bytree parameters next


We will look at 10% intervals from 60% to 100% for both subsample and colsample_bytree

In [ ]:
gamma = gbm.best_params_['gamma']
parameters['gamma'] = gamma
scores.append(gbm.best_score_)

cv_params = {'subsample': [i/10.0 for i in range(6,11)], ### subsample=[0.6, 0.7, 0.8, 0.9, 1.0]
             'colsample_bytree': [i/10.0 for i in range(6,11)] ### colsample_bytree=[0.6, 0.7, 0.8, 0.9, 1.0]
            }

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        max_depth = max_depth,
                                        min_child_weight = min_child_weight,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent

                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)


In [ ]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'colsample_bytree': 0.6, 'subsample': 0.8}

### 5. Refine for subsample and colsample_bytree with a smaller grid of values

Look at 5% intervals in some range around the best values found previously

In [ ]:
subsample = gbm.best_params_['subsample']
colsample_bytree = gbm.best_params_['colsample_bytree']
parameters['subsample'] = subsample
parameters['colsample_bytree'] = colsample_bytree
scores.append(gbm.best_score_)

cv_params = {'subsample': [i/100.0 for i in range(int((subsample-0.1)*100.0), min(int((subsample+0.1)*100),105) , 5)], 
             'colsample_bytree': [i/100.0 for i in range(int((colsample_bytree-0.1)*100.0), min(int((subsample+0.1)*100),105), 5)]
            }

### subsample=[0.7, 0.75, 0.8, 0.85]
### colsample_bytree=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85]

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        max_depth = max_depth,
                                        min_child_weight = min_child_weight,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent

                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)

In [ ]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'colsample_bytree': 0.6, 'subsample': 0.8}

### 6. Tuning reg_alpha and reg_lambda parameters¶

In [ ]:
colsample_bytree = gbm.best_params_['colsample_bytree']
subsample = gbm.best_params_['subsample']
parameters['colsample_bytree'] = colsample_bytree
parameters['subsample'] = subsample
scores.append(gbm.best_score_)

cv_params = {'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100], 
             'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
            }

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        max_depth = max_depth,
                                        min_child_weight = min_child_weight,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        colsample_bytree = colsample_bytree,
                                        subsample = subsample,
                                        silent = silent

                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)


In [ ]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'reg_alpha': 1e-05, 'reg_lambda': 1}

### 7. Refine parameters on a smaller grid

Look at a smaller grid around the best values found previously

In [ ]:
reg_alpha = gbm.best_params_['reg_alpha']
reg_lambda = gbm.best_params_['reg_lambda']
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
scores.append(gbm.best_score_)

cv_params = {'reg_lambda': [reg_alpha*0.2, reg_alpha*0.5, reg_alpha, reg_alpha*2, reg_alpha*5], 
             'reg_alpha': [reg_lambda*0.2, reg_lambda*0.5, reg_lambda, reg_lambda*2, reg_lambda*5]
            }

### reg_lambda= [2.0000000000000003e-06, 5e-06, 1e-05, 2e-05, 5e-05]
### reg_alpha= [0.2, 0.5, 1, 2, 5]

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        max_depth = max_depth,
                                        min_child_weight = min_child_weight,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        colsample_bytree = colsample_bytree,
                                        subsample = subsample,
                                        silent = silent

                                    ),
                   
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)

In [ ]:
print ("Best parameters %s" %gbm.best_params_)

Best parameters {'reg_alpha': 1, 'reg_lambda': 2e-05}

#### At this point, we stopped looking for parameters because we were getting larger RMSE. 



So far, the best model was that used until the third step. That is, a model with:

objective = "reg:linear"
seed = 100
n_estimators = 150
learning_rate = 0.1
gamma = 0.1
subsample = 0.8
colsample_bytree = 0.8
reg_alpha = 1
reg_lambda = 1
silent = False
max_depth = 15
min_child = 7.5

### 8. Lower the learning_rate and set a large num_estimators 

Our last step involves selecting a lower the learning_rate and set a large n_estimators  to ensure convergence.

Here, we choose:

n_estimators= [100,101,102,103,104,105,106,107,108,109,110]

learning_rate = [0.5,0.75,0.1,0.105]

Thus, the XGBOOST model with the best performance (score in kaggle=3458.122) is the following:

In [103]:
objective = "reg:linear"
seed = 100
n_estimators = 105
learning_rate = 0.075
gamma = 0.1
subsample = 0.8
colsample_bytree = 0.8
reg_alpha = 1
reg_lambda = 1
silent = False

parameters = {}
parameters['objective'] = objective
parameters['seed'] = seed
parameters['n_estimators'] = n_estimators
parameters['learning_rate'] = learning_rate
parameters['gamma'] = gamma
parameters['colsample_bytree'] = colsample_bytree
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
parameters['silent'] = silent

scores = []

cv_params = {'max_depth': [15],
             'min_child_weight': [7.5]
            }
gbm = GridSearchCV(xgb.XGBRegressor(objective = objective,
                                    seed = seed,
                                    n_estimators = n_estimators,
                                    learning_rate = learning_rate, 
                                    gamma = gamma,
                                    subsample = subsample,
                                    colsample_bytree = colsample_bytree,
                                    reg_alpha = reg_alpha,
                                    reg_lambda = reg_lambda,
                                    silent = silent,n_jobs=-1),
                    
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 3,
                    verbose = True
)

gbm.fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, importance_type='gain',
       learning_rate=0.075, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=105, n_jobs=-1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1,
       reg_lambda=1, scale_pos_weight=1, seed=100, silent=False,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_depth': [15], 'min_child_weight': [7.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [104]:
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('RMSE:', round(np.sqrt(metrics.mean_squared_error(predictions, y_test)),5))
    return accuracy

In [105]:
best_random = gbm.best_estimator_
random_accuracy = evaluate(gbm, X_test, y_test)

Model Performance
Average Error: 2176.5178 degrees.
Accuracy = 89.24%.
RMSE: 3459.99908


In [106]:
from sklearn.externals import joblib
joblib.dump(gbm, 'gbm_model.pkl', compress=3)

['gbm_model.pkl']

In [107]:
data_test3 = np.array(data_test3)
y_pred = pd.DataFrame(gbm.best_estimator_.predict(data_test3))
y_pred.rename(columns={0:'Price'}, inplace=True)
y_pred

,Price
0,16744.220703
1,15744.460938
2,7719.280762
3,34369.234375
4,9116.747070
5,7134.089355
6,9647.242188
7,17999.208984
8,15406.258789
9,15803.610352


In [108]:
y_pred.to_csv('test_submission.csv', index_label='ID')

############### 

# Exercise P1.2 (50%)

Create an API of the model.


# API on AWS
Following the screnshots from the API hosted on AWS server.

API link:http://18.191.28.236:5000/

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/AP_AWS0.png)

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/AP_AWS1.png)

For use the API directly with the URL, can use:

Example:  
http://18.191.28.236:5000/pricepredict/?year=2018&mileage=1000&state=MD&make=Nissan&model=MuranoAWD

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/AP_AWS2.png)

## API local:

The following images shows the API function on local machine:

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API1.png)


**Now let's try the local API:**

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API2.png)

**The arguments entered:**

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API3.png)

**Run the API:**

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API4.png)

**Show the answer:**

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API5.png)

**Using the API directly with the routhe and parameters:**

![](https://raw.githubusercontent.com/andaoba/MachineLearningCourse/master/Capturas_API_local/API6.png)

In [6]:
############### 